In [5]:
%matplotlib inline
import importlib
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from collections import Counter
import app.lib.sklearn_classification_lib as classification
import app.lib.kafka_util as kafkautil
from datetime import datetime
import json
from  app.lib.elasticsearch_util import Elasticsearch_Util

In [6]:
LISTINGSFILE = 'app/jupyter_test/airbnbdata/listings.csv'
df = pd.read_csv(LISTINGSFILE)
# print df.head(10)
df["@timestamp"] = pd.to_datetime(df["scrape_id"].astype(str))
df["@timestamp"] = pd.to_datetime(df["scrape_id"].astype(str)).dt.strftime('%Y-%m-%d %H:%M:%S')
df_json = df.head(1).to_json(orient='records')
print df_json
# # print df.dtypes
# df_list = json.loads(df_json)

In [3]:
# for i in df_list:
#     print json.dumps(i)
kafkaProducer = kafkautil.create_producer()
kafkautil.produce_kafka_dataframe(kafkaProducer, 'linkdata__machinelearning__node__airbnb', df)

In [7]:
es_util = Elasticsearch_Util()
es_index_name = "machinelearning*"
es_type_name = "airbnb"

query_column_data = {"query": {
                            "match_all": {}
                      }
                    }

es_util.es_read_scroll_scan(es_index_name, es_type_name, query_column_data,500)